In [157]:
import pandas as pd
import numpy as np

# read in file
df = pd.read_csv('census.csv')

# reassign header row
df.columns = df.iloc[0]
df = df[1:]

# filter out columns
col_names = df.columns.tolist()
col_names = [x for x in col_names if "Percent" in x and "Margin" not in x and "Total" not in x]
df = df[col_names]

In [158]:
# clean up column names
indices = [df.columns.tolist()[x].find('!!') for x in range(len(df.columns.tolist()))]
col_names = [df.columns.tolist()[x][indices[x]+2:] for x in range(len(indices))]
dict_colnames = {}
for d in range(len(df.columns.tolist())):
    dict_colnames[df.columns.tolist()[d]] = col_names[d]
df.rename(columns=dict_colnames,
          inplace=True)
df = df.loc[:,~df.columns.duplicated()]
# replace all "-"
df[0:] = df[0:].replace('-', '', regex=True)
df[0:] = df[0:].replace("(X)", '')

In [161]:
# Do linear regression
dict_final = {'x_column':[],'y_column':[],'r_squared':[]}
from sklearn.linear_model import LinearRegression
for x in df.columns.tolist():
    for y in df.columns.tolist():
        if x == y:
            pass
        else:
            x_index = x.find('!!')
            y_index = y.find('!!')
            if x[0:x_index] == y[0:y_index]:
                pass
            else:
                df_temp = df[(df[x]!='')&(df[y]!='')]
                x1 = df_temp[x].values
                x1 = x1.reshape(len(x1), 1)
                y1 = df_temp[y].values
                y1 = y1.reshape(len(y1), 1)
                if len(x1) == 0 or len(y1) == 0:
                    pass
                else:
                    model = LinearRegression()
                    modelfit = model.fit(x1, y1)
                    dict_final['x_column'].append(x)
                    dict_final['y_column'].append(y)
                    dict_final['r_squared'].append(model.score(x1, y1))

In [162]:
df_output = pd.DataFrame(dict_final,columns=['x_column','y_column','r_squared']).sort_values("r_squared",ascending=False)
df_output.to_excel('test.xlsx',index=False)